## HW3: Decision Tree and Random Forest
In hw3, you need to implement decision tree and random forest by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get no points by simply calling sklearn.tree.DecisionTreeClassifier

In [ ]:
!pip install sklearn

## Load data

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
import operator
import re

In [2]:
data = load_breast_cancer()
feature_names = data['feature_names']
print(feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [3]:
x_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
x_test = pd.read_csv("x_test.csv")
y_test = pd.read_csv("y_test.csv")
# print(x_train.shape)

## Question 1
Gini Index or Entropy is often used for measuring the “best” splitting of the data. Please compute the Entropy and Gini Index of provided data. Please use the formula from [page 7 of hw3 slides](https://docs.google.com/presentation/d/1ish3jEr_6be0FK4kgOZa12nYAyJFh0P2LCNsNPOCiXo/edit#slide=id.g7703d1636d_0_21)

In [ ]:
def gini(sequence):
    element1 = np.argmax(np.bincount(sequence))
    size = np.shape(sequence)[0]
    result = 1 - ((sequence.count(element1)/size)**2+((size-sequence.count(element1))/size)**2)
    return result

In [ ]:
def entropy(sequence):
    element1 = np.argmax(np.bincount(sequence))
    size = np.shape(sequence)[0]
    count1 = sequence.count(element1)
    count2 = size-count1
    if count1==0 or count2 == 0:
        return 0
    result = -(count1/size)*np.log2(count1/size)-(count2/size)*np.log2(count2/size)
    return result

In [ ]:
# 1 = class 1,
# 2 = class 2
data = np.array([1,2,1,1,1,1,2,2,1,1,2])
data = np.ones(6)
a =list(data)

In [ ]:
print("Gini of data is ", gini(a))

In [ ]:
print("Entropy of data is ", entropy(a))

## Question 2
Implement the Decision Tree algorithm (CART, Classification and Regression Trees) and trained the model by the given arguments, and print the accuracy score on the test data. You should implement two arguments for the Decision Tree algorithm
1. **Criterion**: The function to measure the quality of a split. Your model should support “gini” for the Gini impurity and “entropy” for the information gain. 
2. **Max_depth**: The maximum depth of the tree. If Max_depth=None, then nodes are expanded until all leaves are pure. Max_depth=1 equals to split data once


In [68]:
class DecisionTree():
    def __init__(self, criterion='gini', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth

        self.DT = self.create_tree(x_train, y_train, max_depth)
        self.prediction = self.predict_label(self.DT, x_test)
        self.accuary = self.evaluation(self.prediction, y_test)

        return None

    def gini(self, sequence):
        element1 = np.argmax(np.bincount(sequence))
        size = np.shape(sequence)[0]
        result = 1 - ((sequence.count(element1) / size) ** 2 + ((size - sequence.count(element1)) / size) ** 2)
        return result

    def entropy(self, sequence):
        element1 = np.argmax(np.bincount(sequence))
        size = np.shape(sequence)[0]
        count1 = sequence.count(element1)
        count2 = size - count1
        if count1 == 0 or count2 == 0:
            return 0
        result = -(count1 / size) * np.log2(count1 / size) - (count2 / size) * np.log2(count2 / size)
        return result

    def find_threshold(self, x_data, y_data):
        best_criterion = 100
        best_attribute = 'bb'
        best_thershold = 0
        feature = list(x_data.columns)
        for _ in feature:
            column = x_data[_]
            sorted_column = np.sort(column)
            for i in range(column.shape[0] - 1):
                average = (sorted_column[i] + sorted_column[i + 1]) / 2
                left, right = list(), list()
                for row in range(column.shape[0]):
                    if column.iloc[row] <= average:
                        left.append(y_data['0'].iloc[row])
                    else:
                        right.append(y_data['0'].iloc[row])
                if self.criterion == 'gini':
                    temp_criterion = len(left) / column.shape[0] * self.gini(left) + len(right) / column.shape[0] * self.gini(right)
                    # print('gini: ', temp_criterion)
                else:
                    temp_criterion = len(left) / column.shape[0] * self.entropy(left) + len(right) / column.shape[0] * self.entropy(right)
                    # print('not gini: ', temp_criterion)
                if temp_criterion < best_criterion:
                    # print('current best: ', temp_criterion)
                    best_criterion = temp_criterion
                    best_attribute = _
                    best_thershold = average
        left_child = x_data[x_data[best_attribute] <= best_thershold]
        del left_child[best_attribute]
        left_y = y_data[x_data[best_attribute] <= best_thershold]
        right_child = x_data[x_data[best_attribute] > best_thershold]
        del right_child[best_attribute]
        right_y = y_data[x_data[best_attribute] > best_thershold]

        return left_child, left_y, right_child, right_y, best_attribute, best_thershold,best_criterion, y_data.shape[0]

    # 在dataSet中返回数目最多的类别标记
    def majorityClass(self, dataSet):
        counts = {}
        labels = dataSet.iloc[:, -1]
        for one in labels:
            if one not in counts.keys():
                counts[one] = 0
            counts[one] += 1
        sortedCounts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
        return sortedCounts[0][0]

    # %%
    def create_tree(self, x_data, y_data, max_depth):
        if 30 - len(x_data.columns) > self.max_depth:
            return self.majorityClass(y_data)
        if len(x_data.columns) == 1:
            return self.majorityClass(y_data)
        if len(set(y_data.iloc[:, -1])) == 1:
            return y_data.iloc[0, -1]
        left_child, left_y, right_child, right_y, bestAttribute, bestTh, bestCriterion, numofInstance = self.find_threshold(x_data, y_data)
        decisionTree = {(bestAttribute,bestCriterion,numofInstance): {}}
        print(decisionTree)
        decisionTree[(bestAttribute,bestCriterion,numofInstance)]['<=' + str(bestTh)] = self.create_tree(left_child, left_y, max_depth)
        decisionTree[(bestAttribute,bestCriterion,numofInstance)]['>' + str(bestTh)] = self.create_tree(right_child, right_y, max_depth)
        return decisionTree

    def classify(self,inputTree,testVec):
        firstStr = list(inputTree)[0]
        secondDict = inputTree[firstStr]
        secondStr = list(secondDict)[0]
        TH = float(re.findall(r"\d+\.?\d*",secondStr)[0])
        attrValue = testVec[firstStr[0]]
        if attrValue <= TH:
            key = '<=' + str(TH)
        else:
            key = '>' + str(TH)
        valueOfFeat = secondDict[key]
        if isinstance(valueOfFeat, dict):
            classLabel = self.classify(valueOfFeat, testVec)
        else: classLabel = valueOfFeat
        return classLabel
    def predict_label(self, tree, test):
        predict = []
        for i in range(test.shape[0]):
            predictLable = self.classify(tree, test.iloc[i, :])
            predict.append(predictLable)
        return predict
    def evaluation(self, predict, y):
        right =0
        for i in range(len(predict)):
            if predict[i] == int(y.iloc[i]):
                right += 1
        print(right)
        return right / len(predict)

        

### Question 2.1
Using Criterion=‘gini’, showing the accuracy score of test data by Max_depth=3 and Max_depth=10, respectively.


In [69]:
clf_depth3 = DecisionTree(criterion='gini', max_depth=3)

{('worst radius', 0.13485126330395825, 426): {}}
{('worst concave points', 0.06702518547717266, 283): {}}
{('area error', 0.011920190723051742, 251): {}}
{('worst texture', 0.007559650366170559, 249): {}}
{('mean radius', 0.0, 2): {}}
{('worst texture', 0.24215686274509804, 32): {}}
{('worst area', 0.2212121212121213, 15): {}}
{('mean perimeter', 0.0, 17): {}}
{('mean texture', 0.06314897223988139, 143): {}}
{('mean concavity', 0.0, 11): {}}
{('mean concave points', 0.015035854730511295, 132): {}}
{('worst concavity', 0.010178117048346055, 131): {}}
133


In [71]:
clf_depth3.accuary

0.9300699300699301

In [72]:
clf_depth10 = DecisionTree(criterion='gini', max_depth=10)

{('worst radius', 0.13485126330395825, 426): {}}
{('worst concave points', 0.06702518547717266, 283): {}}
{('area error', 0.011920190723051742, 251): {}}
{('worst texture', 0.007559650366170559, 249): {}}
{('mean texture', 0.058823529411764705, 17): {}}
{('mean radius', 0.0, 2): {}}
{('mean radius', 0.0, 2): {}}
{('worst texture', 0.24215686274509804, 32): {}}
{('worst area', 0.2212121212121213, 15): {}}
{('mean texture', 0.0, 11): {}}
{('mean radius', 0.0, 4): {}}
{('mean perimeter', 0.0, 17): {}}
{('mean texture', 0.06314897223988139, 143): {}}
{('mean concavity', 0.0, 11): {}}
{('mean concave points', 0.015035854730511295, 132): {}}
{('worst concavity', 0.010178117048346055, 131): {}}
{('mean smoothness', 0.0, 3): {}}
132


In [73]:
clf_depth10.accuary

0.9230769230769231

### Question 2.2
Using Max_depth=3, showing the accuracy score of test data by Criterion=‘gini’ and Criterion=’entropy’, respectively.


In [ ]:
clf_gini = DecisionTree(criterion='gini', max_depth=3)
clf_entropy = DecisionTree(criterion='entropy', max_depth=3)

- Note: All of your accuracy scores should over 0.9
- Note: You should get the same results when re-building the model with the same arguments,  no need to prune the trees
- Hint: You can use the recursive method to build the nodes


## Question 3
Plot the [feature importance](https://sefiks.com/2020/04/06/feature-importance-in-decision-trees/) of your Decision Tree model. You can get the feature importance by counting the feature used for splitting data.

- You can simply plot the feature counts for building tree without normalize the importance

![image](https://i2.wp.com/sefiks.com/wp-content/uploads/2020/04/c45-fi-results.jpg?w=481&ssl=1)

## Question 4
implement the Random Forest algorithm by using the CART you just implemented from question 2. You should implement two arguments for the Random Forest.

1. **N_estimators**: The number of trees in the forest. 
2. **Max_features**: The number of random select features to consider when looking for the best split
3. **Bootstrap**: Whether bootstrap samples are used when building tree


In [6]:
from random import randrange

In [75]:
class RandomForest():
    def __init__(self, n_estimators, max_features, bootstrap=True, criterion='gini', max_depth=None):
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.bootstrap = bootstrap
        self.RM = self.create_forest(self.n_estimators, self.max_features, self.bootstrap)
        self.prediction = self.predict_label(self.RM, x_test)
        self.accuary = self.evaluation(self.prediction, y_test)
        return None
        # return accuracy
    def gini(self, sequence):
        if len(sequence)==0:
            return 0
        element1 = np.argmax(np.bincount(sequence))
        size = np.shape(sequence)[0]
        result = 1 - ((sequence.count(element1) / size) ** 2 + ((size - sequence.count(element1)) / size) ** 2)
        return result
    def create_forest(self,n_estimators,max_features,bootstrap):
        tree_list = []
        for i in range(n_estimators):
            if bootstrap==True:
                x_data,y_data = self.getsubsample(x_train,y_train)
                DT = self.create_tree(max_features,x_data,y_data)
            else:
                DT = self.create_tree(max_features,x_train,y_train)
            tree_list.append(DT)
        return tree_list
    # 在dataSet中返回数目最多的类别标记
    def majorityClass(self, dataSet):
        counts = {}
        labels = dataSet.iloc[:, -1]
        for one in labels:
            if one not in counts.keys():
                counts[one] = 0
            counts[one] += 1
        sortedCounts = sorted(counts.items(), key=operator.itemgetter(1), reverse=True)
        return sortedCounts[0][0]
    def create_tree(self,max_features,x_data,y_data):
        if len(x_data.columns) == 1:
            return self.majorityClass(y_data)
        if len(set(y_data.iloc[:, -1])) == 1:
            return y_data.iloc[0, -1]
        left_child, left_y, right_child, right_y, bestAttribute, bestTh = self.find_threshold(x_data, y_data,max_features)
        decisionTree = {bestAttribute: {}}
        print(decisionTree)
        decisionTree[bestAttribute]['<=' + str(bestTh)] = self.create_tree(max_features,left_child, left_y)
        decisionTree[bestAttribute]['>' + str(bestTh)] = self.create_tree(max_features,right_child, right_y)
        return decisionTree
    def getsubsample(self,x_data,y_data):
        x_columns = x_data.columns
        y_columns = y_data.columns
        subdataset_x = pd.DataFrame(columns=x_columns)
        subdataset_y = pd.DataFrame(columns=y_columns)
        len_data = x_data.shape[0]
        while len(subdataset_x) < len_data:
            index = randrange(x_data.shape[0]-1)
            subdataset_x = subdataset_x.append(x_data.iloc[index,:])
            subdataset_y = subdataset_y.append(y_data.iloc[index,:])
        return subdataset_x,subdataset_y
    def extract_feature(self,feature,max_feature):
        subset = []
        while len(subset)<max_feature:
            index = randrange(len(feature)-1)
            if feature[index] not in subset:
                subset.append(feature[index])
        return subset
    def find_threshold(self, x_data, y_data,max_feature):
        best_criterion = 100
        best_attribute = 'bb'
        best_thershold = 0
        feature = list(x_data.columns)
        feature = self.extract_feature(feature,max_feature)
        for _ in feature:
            column = x_data[_]
            sorted_column = np.sort(column)
            for i in range(column.shape[0] - 1):
                average = (sorted_column[i] + sorted_column[i + 1]) / 2
                left, right = list(), list()
                for row in range(column.shape[0]):
                    if column.iloc[row] <= average:
                        left.append(y_data['0'].iloc[row])
                    else:
                        right.append(y_data['0'].iloc[row])
                # print('length of left',len(left), 'right ', len(right))
                temp_criterion = len(left) / column.shape[0] * self.gini(left) + len(right) / column.shape[0] * self.gini(right)
                if temp_criterion < best_criterion:
                    best_criterion = temp_criterion
                    best_attribute = _
                    best_thershold = average
        left_child = x_data[x_data[best_attribute] <= best_thershold]
        del left_child[best_attribute]
        left_y = y_data[x_data[best_attribute] <= best_thershold]
        right_child = x_data[x_data[best_attribute] > best_thershold]
        del right_child[best_attribute]
        right_y = y_data[x_data[best_attribute] > best_thershold]

        return left_child, left_y, right_child, right_y, best_attribute, best_thershold
    def classify(self,inputTree,testVec):
        firstStr = list(inputTree)[0]
        secondDict = inputTree[firstStr]
        secondStr = list(secondDict)[0]
        TH = float(re.findall(r"\d+\.?\d*",secondStr)[0])
        attrValue = testVec[firstStr]
        if attrValue <= TH:
            key = '<=' + str(TH)
        else:
            key = '>' + str(TH)
        valueOfFeat = secondDict[key]
        if isinstance(valueOfFeat, dict):
            classLabel = self.classify(valueOfFeat, testVec)
        else: classLabel = valueOfFeat
        return classLabel
    def predict_label(self, forest, test):
        predict = []
        for i in range(test.shape[0]):
            daixuan = []
            for j in range(self.n_estimators):
                predictLable = self.classify(forest[j], test.iloc[i, :])
                daixuan.append(predictLable)
            element1 = np.argmax(np.bincount(daixuan))
            predict.append(element1)
        return predict
    def evaluation(self, predict, y):
        right =0
        for i in range(len(predict)):
            if predict[i] == int(y.iloc[i]):
                right += 1
        return right / len(predict)
    


### Question 4.1
Using Criterion=‘gini’, Max_depth=None, Max_features=sqrt(n_features), showing the accuracy score of test data by n_estimators=10 and n_estimators=100, respectively.


In [76]:
clf_10tree = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))

{'worst area': {}}
{'worst concave points': {}}
{'mean area': {}}
{'worst symmetry': {}}
{'worst perimeter': {}}
{'perimeter error': {}}
{'texture error': {}}
{'area error': {}}
{'area error': {}}
{'mean concave points': {}}
{'mean texture': {}}
{'concave points error': {}}
{'perimeter error': {}}
{'mean texture': {}}
{'mean concavity': {}}
{'mean texture': {}}
{'worst area': {}}
{'worst concave points': {}}
{'worst smoothness': {}}
{'worst perimeter': {}}
{'radius error': {}}
{'worst texture': {}}
{'mean concave points': {}}
{'mean perimeter': {}}
{'compactness error': {}}
{'mean texture': {}}
{'worst symmetry': {}}
{'mean perimeter': {}}
{'mean perimeter': {}}
{'concavity error': {}}
{'mean texture': {}}
{'worst concavity': {}}
{'mean concave points': {}}
{'mean radius': {}}
{'worst radius': {}}
{'worst smoothness': {}}
{'texture error': {}}
{'worst area': {}}
{'mean concavity': {}}
{'area error': {}}
{'mean symmetry': {}}
{'worst perimeter': {}}
{'worst radius': {}}
{'worst smoothne

In [78]:
# clf_10tree.predict_label(clf_10tree, x_test)
# print(clf_10tree.evaluation(clf_10tree.prediction, y_test))
clf_10tree.accuary

0.9440559440559441

In [ ]:
clf_100tree = RandomForest(n_estimators=100, max_features=np.sqrt(x_train.shape[1]))

{'worst perimeter': {}}
{'mean concave points': {}}
{'mean symmetry': {}}
{'perimeter error': {}}
{'worst symmetry': {}}
{'worst area': {}}
{'worst texture': {}}
{'worst compactness': {}}
{'smoothness error': {}}
{'mean area': {}}
{'concave points error': {}}
{'area error': {}}
{'mean texture': {}}
{'texture error': {}}
{'mean symmetry': {}}
{'worst area': {}}
{'worst concave points': {}}
{'worst area': {}}
{'worst texture': {}}
{'mean area': {}}
{'radius error': {}}
{'worst concavity': {}}
{'perimeter error': {}}
{'mean texture': {}}
{'texture error': {}}
{'texture error': {}}
{'worst concavity': {}}
{'perimeter error': {}}
{'mean texture': {}}
{'concave points error': {}}
{'mean perimeter': {}}
{'worst radius': {}}
{'mean concave points': {}}
{'worst texture': {}}
{'mean texture': {}}
{'worst area': {}}
{'mean symmetry': {}}
{'mean texture': {}}
{'worst texture': {}}
{'worst concave points': {}}
{'area error': {}}
{'worst symmetry': {}}
{'mean texture': {}}
{'worst concave points': {

In [ ]:
clf_100tree.accuracy

### Question 4.2
Using Criterion=‘gini’, Max_depth=None, N_estimators=10, showing the accuracy score of test data by Max_features=sqrt(n_features) and Max_features=n_features, respectively.


In [ ]:
clf_all_features = RandomForest(n_estimators=10, max_features=np.sqrt(x_train.shape[1]))

In [ ]:

clf_random_features = RandomForest(n_estimators=10, max_features=x_train.shape[1])

- Note: Use majority votes to get the final prediction, you may get slightly different results when re-building the random forest model

## Supplementary
If you have trouble to implement this homework, TA strongly recommend watching [this video](https://www.youtube.com/watch?v=LDRbO9a6XPU), which explains Decision Tree model clearly. But don't copy code from any resources, try to finish this homework by yourself! 